### 1.0 Libraries and directories

In [1]:
import os
import glob
import pprint as pp
import pandas as pd
import geopandas as gpd
import ee
import geemap

ee.Authenticate()
ee.Initialize()

In [2]:
os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')

rois_pattern = f'./data/ew_rois/*_bbox.shp'
rois_file_list = glob.glob(rois_pattern)

rois = [gpd.read_file(file) for file in rois_file_list]
combined_rois = gpd.GeoDataFrame(pd.concat(rois, ignore_index=True))
combined_rois.crs = rois[0].crs


roi_names = combined_rois['roi'].to_list()

print(roi_names)

gswo_monthly_recurrence = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
june = 6
aug = 8
june_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', june)).first()
aug_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', june)).first()

['TUK_MRD', 'YKF', 'YKD', 'AKCP']


In [3]:
def export_rois(gdf, target_img, month_name, roi_name):

    roi = gdf[gdf['roi'] == roi_name]
    # weird geopandas attribute returning a python data structure as a GeoJSON feature collection 
    roi_json = roi.geometry.iloc[0].__geo_interface__
    roi_coords = roi_json['coordinates']
    bounds = ee.Geometry.Polygon(roi_coords)

    clipped_img = target_img.clip(bounds)

    export_params = {
        'image': clipped_img,
        'description': f'GSWORecurrence_{roi_name}_{month_name}', 
        'fileNamePrefix': f'GSWORecurrence_{roi_name}_{month_name}',
        'folder': 'gswo_exports', 
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': bounds,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    }

    ee.batch.Export.image.toDrive(**export_params).start()
        
    

In [4]:
datasets = [june_data, aug_data]
month_names = ['june', 'aug']

monthly_datasets = list(zip(month_names, datasets))

for month, ds in monthly_datasets:
    for roi_name in roi_names:
        export_rois(combined_rois, ds, month, roi_name)